In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import seaborn as sns

In [2]:
df_preprocessed = pd.read_csv("data\dataset_knn_neighbors.csv",  encoding='utf-8')

In [3]:
X = df_preprocessed.drop(columns=['price'])

In [4]:
y = df_preprocessed['price']

In [5]:
numerical_cols_X = df_preprocessed.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_cols_X.remove('price')


In [ ]:

from sklearn.preprocessing import  StandardScaler
from sklearn.model_selection import train_test_split


X_train_g, X_test_g, y_train_g, y_test_g = train_test_split(X, y, test_size=0.2, random_state=42)


X_train_scaled = X_train_g.copy()
X_test_scaled = X_test_g.copy()

scaler = StandardScaler()

X_train_scaled[numerical_cols_X] = scaler.fit_transform(X_train_g[numerical_cols_X])
X_test_scaled[numerical_cols_X] = scaler.transform(X_test_g[numerical_cols_X])



# XGBoost

In [7]:
import xgboost as xg
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

xgb_r = xg.XGBRegressor(objective ='reg:linear',
                  n_estimators = 1000, seed = 123)

# Fitting the model
xgb_r.fit(X_train_scaled, y_train_g)
print("TRAINING SET: ")
print("R² Score :", xgb_r.score(X_train_scaled, y_train_g))

# Predict the model
pred = xgb_r.predict(X_test_scaled)

# RMSE Computation
print("TEST SET:")
print("Mean Squared Error:", mean_squared_error(y_test_g, pred))
print("Mean Absolute Error:", mean_absolute_error(y_test_g, pred))
print("R² Score :", r2_score(y_test_g, pred))

c:\Users\herms\Desktop\BeCode-Bauman\TheHill_Projects\immo-data-regression\challenge-regression\venv\lib\site-packages\xgboost\training.py:183: UserWarning: [09:30:22] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  bst.update(dtrain, iteration=i, fobj=obj)


TRAINING SET: 
R² Score : 0.9393704169950872
TEST SET:
Mean Squared Error: 5166386500.092345
Mean Absolute Error: 50471.37032604133
R² Score : 0.8089058025581726


#### Features Importance

In [8]:
feature_names = X_train_scaled.columns.tolist()

In [11]:
importances = xgb_r.feature_importances_
feature_importance_xg = pd.Series(importances, index=feature_names).sort_values(ascending=False)

print("XGBoost Feature Importances:")
print(feature_importance_xg)

XGBoost Feature Importances:
postCode_8300        0.049130
province_Hainaut     0.042796
province_Brussels    0.039214
province_Liège       0.035776
province_Namur       0.025060
                       ...   
postCode_6441        0.000000
postCode_6531        0.000000
postCode_6461        0.000000
postCode_6593        0.000000
type_HOUSE           0.000000
Length: 1162, dtype: float32


### Actual vs predicted

In [ ]:
plt.scatter(y_test_g, pred)
plt.plot([y_test_g.min(), y_test_g.max()], [y_test_g.min(), y_test_g.max()], c='g', lw=2)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs Predicted')
plt.show()

### Residuals

In [ ]:
residuals_g= y_test_g - pred

plt.scatter(pred, residuals_g)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Predicted')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted')
plt.show()

In [ ]:
sns.histplot(residuals_g, kde=True)
plt.title('Distribution of Residuals')
plt.xlabel('Residuals')
plt.show()